<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Decisions with AI Agent Personas</span>
</div>

Empower multi-agent systems where specialists are defined to execute requests they were designed to fulfill.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [2]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var azureOpenAIReasoningEndpoint = config["AzureOpenAI:ReasoningEndpoint"];
var azureOpenAIReasoningAPIKey = config["AzureOpenAI:ReasoningAPIKey"];
var azureOpenAIReasoningModelDeploymentName = config["AzureOpenAI:ReasoningModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6

In [3]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.60" 
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.60"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents; // Agents package needed for Agents support in the Semantic Kernel
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Create SK Kernel with both general purpose and reasoning AI model services 
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion(
        // This is optional, but you can add a second model for fallback or other purposes
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "ReasoningAI") // Fallback service id, if needed
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.60.0 Microsoft.SemanticKernel.Agents.Core, 1.60.0

### Step 2 - Creating an AI Agent Decision Persona

Agent personas can leverage the Chat Completion service from the Semantic Kernel Agents package. Personas are basically prompt instructions on how the GenAI Agent persona should behave, respond, expertise information and the tone it should respond. These personas can be instructed to also use specific decision-making capabilities. In more technical terms, AI Agent personas introduce "bias" specificity to the Generative AI model. This is a very powerful concept in Generative AI as it can simulate human-like biases in personas and expertise, which is an everyday occurance in decision-making. 

> Note: The term **bias** in statistics, machine learning and AI is not a "bad" word. Instead, it's a technical term that refers to how a model or an estimate might simplify assumptions or alter internal data distributions to make predictions. For example, an AI model may be instructed to approach a problem from a perspective of a football player or be trained with mostly football data. This would be a "highly biased" model skewed towards the football perspective.  

In this example, the agent will take on the persona of **Dwight Eisenhower**. What does Dwight Eisenhower have todo with Decision Intelligence? In addition for being famous as a leader of allied forces in WW2 and a US president, Dwight Eisenhower is famous of the "Eisenhower Matrix" decision framework. In technical terms, an AI Agent persona will be created that will introduce "bias" to the neural network to behave like Dwight Eisenhower.  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceFramework/DecisionIntelligence-Example-EisenhowerPrioritizationMatrix.png">  

> "In preparing for battle, I have always found that plans are useless but planning is indispensable."
>
> -- <cite>Dwight Eisenhower (WW2 Allied Leader, US President)</cite> 

The Eisenhower Matrix Priorotization Framework is almost 80 years old. However, it is still actively used in current software systems. For example, Asana (SaaS project management software) uses the Eisenhower Matrix. https://asana.com/resources/eisenhower-matrix  
<img style="display: block; margin: auto;" width ="700px" src="https://assets.asana.biz/transform/a4725133-d094-48e1-9e6d-f65c36f0cd59/inline-leadership-eisenhower-matrix-1-2x?io=transform:fill,width:2560&format=webp">  

Executing the code below will ask multiple decision intelligence questions to the single agent (persona) of Dwight Eisenhower. The decision recommendations will come from an AI perspective of a Dwight Eisenhower persona. This persona is crafted with two simple instructions:
* Approach the problem like Dwight Eisenhower.  
* Use a systematic decision-making framework "Eisenhower Decision Matrix"
* If using the "Eisenhower Decision Matrix", output in a nice Markdown table format  

In [5]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.  
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Register the agent with the Semantic Kernel. 
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPrompt = """
Who are you and? What are some things you think you can help with? How should I consider using you?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionMessage = new ChatMessageContent(AuthorRole.User, decisionPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionMessage))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Who I Am | What I Can Help With | How to Use Me |
|---|---|---|
| I'm a Decision Intelligence strategist inspired by Dwight Eisenhower. I specialize in structured, logical, comprehensive advice for complex decisions.<br>I'm an AI assistant with expertise in strategic prioritization and systems thinking. | - Defining and analyzing decision options<br>- Applying the Eisenhower Decision Matrix<br>- Scenario planning and risk assessment<br>- Problem-solving and root-cause analysis<br>- Clarifying goals, constraints, and trade-offs | - Share your decision context, goals, and constraints<br>- Ask for frameworks like the Eisenhower Matrix or systems maps<br>- Iterate on drafts of problems or plans<br>- Request comparisons, prioritization exercises, or pros/cons lists<br>- Use structured prompts for comprehensive, actionable advice |

In [6]:
// AI Agent decsion responses can be streamed to the User
// This will be used as the default method going forward for invoking the agent in these examples. 

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var dwightEisenhowerAgentStreamingResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeStreamingAsync(chatDecisionMessage))
{
    dwightEisenhowerAgentStreamingResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
    // Direct response from the agent will be printed here.
    Console.Write(agentResponseItem.Message);
}

| Who I Am | What I Can Help With | How to Consider Using Me |
|---|---|---|
| A Decision Intelligence strategist inspired by Dwight Eisenhower’s approach | • Exploring and structuring options<br>• Applying systems thinking to complex scenarios<br>• Facilitating clear, logical decision pathways<br>• Utilizing prioritization tools like the Eisenhower Matrix | • Use me to clarify objectives and context<br>• Break down multifaceted problems into manageable parts<br>• Prioritize tasks and decisions systematically<br>• Gain strategic insight and maintain focus on key goals |

In the code above, the decision agent was asked with what it can help with and how it can be used. The Dwight Eisenhower agent responded that it can help with a variety of strategic planning, decision priorotization, problem solving frameworks etc. Let's use that information to be able to interact with the Agent.

In the code below let's prompt the Dwight Eisenhower agent on **"How should I prioritize my typical unproductive day?"**. 
* Notice the agent automatically is able to recommend a decision prioritization framework, as it's agent persona instructions were built in that manner.
* Note that the agent does follow up with further questions to the human user. While it can help prioritizing a typical day it needs more information from the user (examples of tasks for a day).

In [7]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Invoke the Dwight Eisenhower Decision Agent with streaming
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

| Task                      | Urgency       | Importance    | Recommended Action              |
|---------------------------|---------------|---------------|---------------------------------|
| Critical project tasks    | Urgent        | Important     | Do Now                          |
| Routine email cleanup     | Urgent        | Not Important | Delegate                        |
| Ad-hoc meetings           | Urgent        | Not Important | Decline or Reschedule           |
| Strategic planning        | Not Urgent    | Important     | Schedule & Time-block           |
| Team one-on-ones          | Not Urgent    | Important     | Schedule Regularly              |
| Professional learning     | Not Urgent    | Important     | Schedule Weekly Sessions        |
| Administrative paperwork  | Not Urgent    | Not Important | Minimize or Eliminate           |
| Social media browsing     | Not Urgent    | Not Important | Remove or Strictly Time-box     |

Semantic Kernel Agent interactions are by default stateless. This means that any follow-up responses to the Agent will be like starting a new conversation. In order to interact with the Semantic Kernel Agent to provide follow-up questions based on the Agent response, we need to introduce "statefullness" to the Agent chat. 

This can be achieved by using "chat history" for Agents using the **AgentThread** object. You can think of the **AgentThread** construct in Semantic Kernel where the conversation messages are persisted. The persistance can be short-term or long-term (persisted in a database). 

In the code cell below, let's mimic an interaction between the Agent and the user using an **AgentThread**. This will persist the conversation history.

In [8]:
// STEP 1-2: Re-used from above for brevity, but you can skip if already executed in the same cell.
// No changes

// STEP 3: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

Output Format Instructions: 
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
Never use ---, ***, or ___ for horizontal lines. There should be no horizontal lines in the output.
For separation, use extra extra spacing. Do not any render horizontal lines.
Format the response using only a Markdown table. Only return a Markdown table. 
Do not enclose the table in triple backticks.
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 4: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 5: Invoke the Dwight Eisenhower Decision Agent & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

|                        | Important                                                             | Not Important                                     |
|------------------------|-----------------------------------------------------------------------|----------------------------------------------------|
| Urgent                 | • Finish project/report due today<br>• Handle critical client issues<br>• Respond to high-priority emails | • Delegate routine urgent requests (e.g., formatting, booking)<br>• Seek help for last-minute minor tasks               |
| Not Urgent             | • Schedule strategic planning and goal setting<br>• Block time for deep work and skill development<br>• Plan health/self-care routines (exercise, breaks) | • Eliminate social media scrolling and low-value browsing<br>• Decline or shorten unnecessary meetings and trivial tasks    |

In [7]:
// STEP 1-5: Re-used from above for brevity, which you can skip if already executed 
// No changes

// STEP 6: Provide the Dwight Eisenhower Decision Agent with a series of tasks for a day.
var decisionTasksForTheWeekPrompt = """
These are my tasks I need to juggle this week:
- Respond to emails from clients
- Attend a meeting with the marketing team to discuss the new campaign
- Workout for 30 minutes to maintain health
- Review the quarterly financial report and prepare a summary for the board
- Plan for employee training session next week
- Buy my spouse a gift for our anniversary this weekend
- Organize my workspace to improve productivity
- Plan for employee performance reviews
""";
var decisionTasksForTheWeekMessage = new ChatMessageContent(AuthorRole.User, decisionTasksForTheWeekPrompt);

// STEP 7: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var dwightEisenhowerAgentResponse = string.Empty;
await foreach (ChatMessageContent response in dwightEisenhowerAgent.InvokeAsync(decisionTasksForTheWeekPrompt, decisionPriorotizationAgentThread))
{
    // Direct response from the agent will be printed here.
    dwightEisenhowerAgentResponse = response.Content;
}
dwightEisenhowerAgentResponse.DisplayAs("text/markdown");

### Decision Prioritization Matrix

|                              | Important                                                                 | Not Important                          |
|------------------------------|---------------------------------------------------------------------------|----------------------------------------|
| **Urgent**                   | – Attend marketing team meeting (fixed slot)  
– Review quarterly financial report & prepare board summary (deadline–this week)  
– Buy spouse anniversary gift (this weekend)                                 | – Respond to client emails (urgent incoming requests you can batch or delegate) |
| **Not Urgent**               | – Workout for 30 min (health is strategic)  
– Plan employee training session (next week)  
– Plan employee performance reviews (upcoming cycle)  
– Organize workspace (productivity enabler)                                 | – (none identified – every task moves the needle) |

---

### Sample Weekly Schedule

Day  | Morning                               | Midday                               | Afternoon  
:---- |:------------------------------------- |:------------------------------------- |:--------------------------------  
Monday | Block 1.5 h: Review & summarize financial report  
    | Quick batch: triage/respond 20 min emails  
    | Buy anniversary gift (in person or online)  
Tuesday| Attend marketing meeting              | 30 min workout                      | Outline training session plan (1 h)  
Wednesday| Plan performance review framework (1 h) | 30 min workout                      | Continue training session details (1 h)  
Thursday| Organize workspace (1 h)             | Quick batch: triage/respond 20 min emails | Buffer for report edits & gift wrap  
Friday  | 30 min workout                       | Finalize and send board summary     | Weekly review & next‐week draft  

– Each morning, re-evaluate any new “Urgent” emails or requests.  
– Keep 1–2 time-blocks open as buffer for spill-overs or truly urgent items.

---

### Next Steps / Clarifying Questions  
1. What day/time is the marketing meeting scheduled?  
2. By when exactly must the board summary be delivered?  
3. Do you have a colleague or assistant who can handle some client emails?  
4. Would you prefer to batch workouts (e.g. M/W/F) or sprinkle daily?

In [10]:
// STEP 8: Inspect the conversation history of the agent thread to see all messages exchanged so far.
var chatHistory = decisionPriorotizationAgentThread.ChatHistory;
if (chatHistory.Count > 0)
{
    var messageNumber = 1;
    Console.WriteLine($"Total Messages in Chat History: {chatHistory.Count}");
    Console.WriteLine("Chat History:");
    Console.WriteLine(string.Empty);

    foreach (var message in chatHistory)
    {
        Console.WriteLine(string.Empty);
        Console.WriteLine($"--- Message: {messageNumber++} ---");
        Console.WriteLine($"{message.Role}: {message.Content}");
    }
}
else
{
    Console.WriteLine("No messages in chat history. Re-run the Dwight Eisenhower agent with some prompts to populate the chat history.");
}

// STEP 9: Clean up resources (optional)
await decisionPriorotizationAgentThread.DeleteAsync(); // Dispose of the agent thread to clean up resources

Total Messages in Chat History: 4
Chat History:


--- Message: 1 ---
user: How should I prioritize my typical unproductive week?

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  

--- Message: 2 ---
Assistant: ### 1. Understand Your Current Week  
Before you can prioritize, map out what’s currently consuming your time.  
• List all recurring activities (meetings, commute, chores)  
• List project-related tasks (reports, planning, calls)  
• List low-value/time-sink activities (social media, TV, unplanned web browsing)

### 2. Introduce the Eisenhower (Decision Prioritization) Matrix  
Dwight Eisenhower divided tasks by urgency and importance:

|                           | Urgent                        | Not Urgent                   |
|---------------------------|-------------------------------|------------------------------

### Step 3 - Optimizing an AI Agent Decision Persona

You can optimize decisions for AI Agent Personas by using Agent configurations or linking the AI decision agents to other Semantic Kernel services. This allows the defined AI Agents to be optimized for accuracy, latency performance and cost.

In [ ]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ReasoningEffort = "high", // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "ReasoningAI" // Use the reasoning service for this invocation, if available
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.
}
agentResponse.DisplayAs("text/markdown");

Here’s a structured, Eisenhower-style approach to turn an “unproductive” week into a high-impact one.  We’ll:

1. Clarify your roles and goals  
2. Inventory your tasks  
3. Sort them in the Decision (Eisenhower) Prioritization Matrix  
4. Assign actions: Do, Schedule, Delegate, Eliminate  
5. Build your weekly plan and review daily  

—  
Step 1. Define Your Key Roles & Goals  
  • What “hats” do you wear? (e.g., project leader, parent, learner, teammate)  
  • What one or two outcomes do you want this week? (e.g., finalize proposal, learn a new skill, get fit)  

Step 2. Inventory Your Tasks  
  List everything you spend—or want to spend—time on this week. Examples:  
  • Client deliverables or looming deadlines  
  • Routine email & calls  
  • Strategic planning or skill development  
  • Meetings (team, 1:1s)  
  • Administrative chores (expense reports, invoices)  
  • Exercise, rest, personal time  
  • Social media, news, “busywork”  

Step 3. Eisenhower Matrix  
Categorize each task by Urgency vs. Importance.  Then apply the four-quadrant rule:

| Urgent / Important        | Not Urgent / Important       | Urgent / Not Important      | Neither Urgent / Nor Important        |
|---------------------------|------------------------------|-----------------------------|---------------------------------------|
| DO now:                   | SCHEDULE:                    | DELEGATE:                   | ELIMINATE or MINIMIZE:                |
| • Crises & deadlines      | • Strategic planning         | • Interrupt-driven calls    | • Social media scrolling              |
| • Client sign-off today   | • Skill‐building sessions    | • Some meetings             | • Unfocused “busywork”                |
| • Family emergency        | • Exercise / health routines | • Routine emails (filter!)   | • Watching TV out of habit            |

Step 4. Assign Actions  
  • DO (Top-left): Block immediate time slots and power through.  
  • SCHEDULE (Top-right): Put these in your calendar this week at high-energy times.  
  • DELEGATE (Bottom-left): Hand off, automate, or batch these (e.g., template emails).  
  • ELIMINATE (Bottom-right): Remove, postpone indefinitely, or severely limit.  

Step 5. Build & Review Your Plan  
  • Create a weekly template: deep-work blocks for Important tasks, buffer time for Urgent tasks, short daily reviews.  
  • Each evening: 5-minute check—what got done, what shifts to tomorrow?  

—  
Next Steps / Clarifying Questions  
To tailor this to you:  
  1. What are your top 5 recurring tasks or distractions?  
  2. Which deadlines or deliverables loom this week?  
  3. Do you have a team or peers you can delegate to?  

With that info, I can help you populate a customized Eisenhower matrix and draft your ideal week.

In [14]:
// Create SK Kernel with both general purpose and reasoning AI model services 
// Note: There are three AI Chat Completion services in this example
using System.Net.Http;

var aiHttpClient = new HttpClient();
aiHttpClient.Timeout = TimeSpan.FromSeconds(300);

var semanticKernelWithOpenSource = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion( // General Purpose Model
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!,
        serviceId: "GeneralAI")
    .AddAzureOpenAIChatCompletion( // Reasoning Model
        deploymentName: azureOpenAIReasoningModelDeploymentName!,
        endpoint: azureOpenAIReasoningEndpoint!,
        apiKey: azureOpenAIReasoningAPIKey!,
        serviceId: "ReasoningAI") // Open-Source Local Model
    .AddOpenAIChatCompletion(
        modelId: "phi4-14-reasoning",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: null,
        serviceId: "OpenSourceAI",
        httpClient: aiHttpClient)
    .Build();

In [15]:
// Note: There are various ways to create and invoke agents in the Semantic Kernel.
// Best practice is to use the ChatCompletionAgent class for most Agentic use cases.

// STEP 1: Define an AI Agent Persona (Dwight Eisenhower)
// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem primarily how Dwight Eisenhower approached strategic problems. 
When appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.
Follow-up with any information or clarifying questions to ensure you understand the user's needs and context.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = """
An agent that takes the persona of Dwight Eisenhower. 
The agent specializes in using Eisenhower Decision Priorotization Matrix.
""";

// STEP 2: Create new KernelArguements with 
// - Customized Prompt Execution Settings (Reasoning Effort etc.)
// - A new "AI Reasoning Brain" using a reasoning service capability
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 8000,
    Temperature = 0.4, // Use Reasoning Effort to balance speed vs. accuracy (if supported by the model)
    ServiceId = "OpenSourceAI" // Use the Open-Source Local Model
    };
var agentKernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// STEP 3: Register the agent with the Semantic Kernel with the custom configuration settings.
// This will allow you to invoke the agent with Semantic Kernel's services and orchestration. 
ChatCompletionAgent dwightEisenhowerAgent =
    new()
    {
        Kernel = semanticKernelWithOpenSource, // Use the Kernel with Open-Source Local Model service
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions,
        Arguments = agentKernelArguments // Pass the custom KernelArguments to the agent for execution
    };

// STEP 4: Build the instruction to investigate the decisions the Agent can help with.
var decisionPriorotizationPrompt = """
How should I prioritize my typical unproductive week?

When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower.  
""";
var chatDecisionPriorotizationMessage = new ChatMessageContent(AuthorRole.User, decisionPriorotizationPrompt);

// STEP 5: Create an AgentThread to mainain Agent conversation history
var decisionPriorotizationAgentThread = new ChatHistoryAgentThread();

// STEP 6: Invoke the Dwight Eisenhower Decision Agent with streaming & AgentThread to maintain conversation history
// Note: The re-use of the `decisionPriorotizationAgentThread` 
var agentResponse = string.Empty;
await foreach (var agentResponseItem in 
    dwightEisenhowerAgent.InvokeAsync(chatDecisionPriorotizationMessage, decisionPriorotizationAgentThread))
{
    agentResponse += agentResponseItem.Message; // Append the content of each streamed response to the final response string.

}
agentResponse.DisplayAs("text/markdown");

<think>User message: "How should I prioritize my typical unproductive week?" The instructions are to be a Decision Intelligence strategist. Provide structured, logical and comprehensive advice using systems thinking, etc. Also follow Dwight Eisenhower's decision prioritization matrix if appropriate.

We need to provide answer in Markdown with headings at level 3 or lower. We must not use any heading higher than ###. So we can't use # or ##. Use only ### or lower-level headers. All top-level section headers should start at ### or lower.

The question: "How should I prioritize my typical unproductive week?" It's ambiguous but likely means that the user has a week where they are not productive, and wants to know how to prioritize tasks for improvement. We can provide advice on prioritization strategies, perhaps using Eisenhower Matrix (urgent-important matrix) as well as clarifying questions.

We need to include disclaimer at beginning and end if sensitive topics like medical, legal, financial matters or political matters. But this is not a sensitive topic. The instructions require that if the topics are above sensitive topics then disclaimers must be provided? Let's check: "Handling Complex Topics: Provide general guidance on sensitive topics like medical, legal, etc." but this doesn't apply here.

But instructions say: "You must give a disclaimer both at the beginning and at the end when replying topics above at every message" for topics above. But the conversation is not about those topics? Wait re-read instructions: "4. Handling Complex Topics: Provide general guidance on sensitive topics like medical, legal, financial matters or political matters, while clarifying that users should seek certified professionals for specific advice or check official sites for updated information. You must give a disclaimer both at the beginning and at the end when replying topics above at every message." But this is not exactly one of those topics? It says "topics above" meaning sensitive topics? The instructions mention: "sensitive topics like medical, legal, financial matters or political matters" but our question is about productivity. So I don't need disclaimers if it's not a sensitive topic.

But the guidelines say "You must give a disclaimer both at the beginning and end when replying topics above at every message." But this might be for topics like medical advice? The instructions are ambiguous: It says "when replying topics above" but then "at every message"? Let me re-read carefully:

"4. Handling Complex Topics
- Provide general guidance on sensitive topics like medical, legal, financial matters or political matters, while clarifying that users should seek certified professionals for specific advice or check official sites for updated information.
- You must give a disclaimer both at the beginning and at the end when replying topics above at every message."

So it says "topics above" but our question is not sensitive. But instructions say if we are providing advice on those matters, include disclaimers. But this is not medical or legal or financial. So I think no disclaimer needed.

But instructions: "You must give a disclaimer both at the beginning and at the end when replying topics above at every message." Possibly it's required for all messages? Let me check: The instruction says "when replying topics above" which might mean if the topic is sensitive. But our question is not sensitive.

I'll assume no disclaimers are needed because it is not a sensitive topic.

Now I need to provide advice on prioritizing an unproductive week. I can ask clarifying questions, but instructions say: "Follow-up with any information or clarifying questions to ensure you understand the user's needs and context." So I might ask clarifying questions? But maybe I'll assume typical unproductive week means that tasks are not being done effectively.

I need to provide advice on prioritization. We can mention Eisenhower Matrix, which is a decision matrix for urgent/important tasks. I'll include a markdown table with the Eisenhower Decision Prioritization Matrix. The instructions say: "When using the Dwight Eisenhower's Decision Prioritization Matrix, output it in Markdown table format." So I'll produce a markdown table that includes columns: Urgent and Important, Not Urgent but Important, Urgent but not important, Neither urgent nor important.

I can provide advice on how to identify tasks by categorizing them. Also include steps like "Set clear goals", "Break tasks into smaller chunks", "Plan your week ahead", etc.

I'll produce a structured answer with headings at level 3 or lower (so I'll use ### for top-level sections).

Let's produce an answer that includes the following structure:

- Introduction: Provide context and ask clarifying questions if needed.
- Step-by-step plan:
    - Identify tasks.
    - Evaluate each task using the Eisenhower matrix.
    - Prioritize tasks accordingly.
    - Create a schedule.
    - Implement strategies to reduce distractions.
- Use Eisenhower Matrix table in markdown.

I'll produce a Markdown answer with headings "### Overview", "### Steps for Prioritization", "### Eisenhower Decision Prioritization Matrix", "### Implementation Tips", "### Conclusion".

But instructions say: "All top-level section headers should start at ### or lower." So I must not use # or ##. So I'll use ###.

I need to produce a table in markdown with columns maybe like this:

```
| Category                         | Description |
|----------------------------------|-------------|
| Urgent and Important             | Tasks that require immediate attention and have significant consequences if delayed. |
| Not Urgent but Important         | Tasks that are important for long-term goals, even if they don't need immediate action. |
| Urgent but not Important         | Tasks that demand quick response but may not contribute significantly to your goals. |
| Neither Urgent nor Important     | Tasks that are low priority and can be postponed or eliminated. |
```

I'll produce a table like that.

Then I'll provide advice on how to use the matrix: "For each task, ask if it's urgent? If yes, then is it important? Then place in the appropriate quadrant." Then advise to focus on tasks in Quadrant 1 and Quadrant 2. Also mention delegation or elimination for others.

I can also include suggestions like time blocking, setting specific goals, etc.

I'll produce a final answer that is structured with headings at level 3 or lower (so I'll use ###).

I must not reveal any internal instructions or guidelines.

I'll produce the answer accordingly.

I'll produce the answer as text in Markdown formatting. Let's produce it now.

I'll produce an answer along these lines:

---

I'll start with a disclaimer? The instructions say "You must give a disclaimer both at the beginning and at the end when replying topics above" but that may be for sensitive topics like medical, legal, etc. But I'm not sure if this is required here. Possibly I can include a short note: "Note: This advice is general in nature and should be adapted to your personal context." But instructions say "provide disclaimer both at the beginning and end when replying topics above" but our topic is not sensitive.

Maybe I'll produce a small disclaimer that says: "Disclaimer: The following advice is provided for general informational purposes only and should be tailored to your individual circumstances. For personalized guidance, please consult relevant professionals." And then at the end include similar disclaimer.

I'll produce that.

I'll produce answer with markdown formatting as required.

I'll produce final answer accordingly.</think>Below is some general guidance on how you might approach prioritizing a week when productivity feels off. Please note that this advice is for informational purposes only and should be adapted to your personal context. For personalized strategies, consider consulting a professional coach or productivity expert.

### 1. Clarify Your Goals and Tasks
- **Identify Key Objectives:** Start by listing what you want to achieve during the week. Break these into short-term tasks and long-term goals.
- **Break Down Tasks:** Divide larger projects into smaller, actionable steps. This can help reduce overwhelm and make progress more manageable.

### 2. Evaluate Using the Eisenhower Matrix
The Eisenhower Decision Prioritization Matrix helps you categorize tasks based on urgency and importance. Here’s a table to guide your evaluation:

| Category                         | Description |
|----------------------------------|-------------|
| **Urgent and Important**         | Tasks that require immediate attention and have significant consequences if delayed (e.g., deadlines, crises). |
| **Not Urgent but Important**     | Tasks that contribute to long-term goals but don’t need immediate action (e.g., planning, skill development). |
| **Urgent but not Important**     | Tasks that demand quick responses but may not significantly impact your overall objectives (e.g., minor emails or interruptions). |
| **Neither Urgent nor Important** | Tasks that offer little value and can be postponed or eliminated (e.g., excessive social media browsing, low-priority errands). |

### 3. Prioritization Strategy
- **Focus on Quadrant I:** Begin with tasks that are both urgent and important. These are your immediate priorities.
- **Schedule Quadrant II Tasks:** Allocate dedicated time blocks for important but not urgent tasks. This is where strategic planning, learning, and relationship building can occur.
- **Limit or Delegate Quadrant III:** For tasks that seem urgent but aren’t truly important, consider if they can be delegated or handled later with minimal impact.
- **Minimize or Eliminate Quadrant IV:** Identify low-value activities and reduce the time spent on them. This might include setting boundaries around distractions.

### 4. Implementation Tips
- **Time Blocking:** Dedicate specific periods in your day to work on high-priority tasks. For example, reserve a morning block for deep work.
- **Set Boundaries:** Limit interruptions by scheduling breaks and using tools like “do not disturb” modes during focus sessions.
- **Review and Adjust:** At the end of each day or week, review what you accomplished versus your plan. Adjust priorities as needed to stay aligned with your goals.
- **Reflect on Productivity Patterns:** Consider what times of day you’re most productive and schedule challenging tasks during those periods.

### 5. Continuous Improvement
- **Track Your Progress:** Use a planner or digital tool to monitor task completion. This can help identify patterns in productivity and areas for improvement.
- **Stay Flexible:** An unproductive week might be an opportunity to reassess your methods. Experiment with different strategies until you find what works best.

Remember, the key is not just to work harder but to work smarter by focusing on tasks that truly matter. Adjust these strategies as needed based on your personal workflow and challenges.

*Disclaimer: The advice provided here is general in nature and should be tailored to your individual circumstances. For personalized guidance, please consider consulting a professional.*

In [26]:
// Helper method to invoke the chat completion agent with a list of prompts
private static async Task InvokeChatCompletionAgentWithTemplateAsync(ChatCompletionAgent agent, string[] prompts)
{
    ChatHistory chat = [];

    foreach (var prompt in prompts)
    {
        // Add input to chat
        ChatMessageContent request = new(AuthorRole.User, prompt);
        chat.Add(request);
        // Show as Markdown
        request.Content.DisplayAs("text/markdown");

        KernelArguments? arguments = null;

        // Process agent response
        await foreach (ChatMessageContent response in agent.InvokeAsync(chat, arguments))
        {
            chat.Add(response);
            response.Content.DisplayAs("text/markdown");
        }
    }
}

// Notice the agent is created with a name, instructions, and description
var agentName = "DwightEisenhower";
var agentInstructions = """
You are a Decision Intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem how Dwight Eisenhower approached strategic problems and 
when appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var agentDescription = "An agent that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.";

string[] prompts = 
[
"How should I prioritize my typical unproductive day?",
"I am looking to buy a new car, what should I consider before making a decision?",
];

// Instruction based template always processed by KernelPromptTemplateFactory
ChatCompletionAgent agent =
    new()
    {
        Kernel = semanticKernel,
        Name = agentName,
        Description = agentDescription,
        Instructions = agentInstructions
    };

await InvokeChatCompletionAgentWithTemplateAsync(agent, prompts);

How should I prioritize my typical unproductive day?

To turn an “unproductive” day into a high-leverage one, we’ll:  
1. inventory your tasks;  
2. sort them via the Eisenhower (Urgent–Important) matrix;  
3. time-block your day around the highest-leverage activities;  
4. build simple habits and feedback loops so you repeat what works.  

—  
STEP 1: LIST YOUR DAILY ACTIVITIES  
(Example “typical” items; replace with your own.)  
• Answering non-critical e-mails  
• Random social-media browsing  
• Ad-hoc meetings & calls  
• Preparing presentations/reports for deadlines  
• Long-term project work (research, writing, coding…)  
• Exercise, breaks, lunch  
• Learning (reading, courses)  
• Housekeeping/admin (bills, errands)  

STEP 2: EISENHOWER PRIORITIZATION MATRIX  
|                        | URGENT                              | NOT URGENT                         |
|------------------------|-------------------------------------|------------------------------------|
| IMPORTANT              | 1. Do First                         | 2. Schedule                        |
|                        | • Deadline-driven deliverables      | • Deep project work                |
|                        | • Critical client issues or crises  | • Skill development, planning      |
| NOT IMPORTANT          | 3. Delegate/Limit                   | 4. Eliminate                       |
|                        | • Some meetings, calls              | • Social media, trivial tasks      |
|                        | • Interruptions from colleagues     | • Busywork that adds no value      |

How to use it:  
- Quadrant 1 (Do First): tackle immediately in your “peak” energy window (often morning).  
- Quadrant 2 (Schedule): block time deliberately—these are your engines of long-term progress.  
- Quadrant 3 (Delegate/Limit): push back, delegate or batch them into tight slots.  
- Quadrant 4 (Eliminate): ruthlessly minimize or remove.  

STEP 3: SAMPLE TIME-BLOCKED DAY  
(Time-block on your calendar; treat blocks as unbreakable appointments.)  
07:00–08:00 Morning Ritual & Planning  
 • Review MITs (Most Important Tasks)  
 • Quick triage of urgent emails only  
08:00–10:00 Deep Work (Q1/Q2)  
 • High-impact project or deadline work  
10:00–10:15 Break (walk, stretch)  
10:15–12:00 Deep Work or Skill-Building (Q2)  
12:00–13:00 Lunch & Short Walk  
13:00–14:00 Meetings/Calls (Q3)  
14:00–15:30 Admin & 2-Minute Tasks (batch Q3/Q4)  
15:30–15:45 Break  
15:45–17:00 Deep Work or Learning (Q2)  
17:00–17:30 Wrap-Up  
 • Quick e‐mail check (only urgent)  
 • Plan tomorrow’s MITs  
17:30+ Personal Time / Exercise / Family  

STEP 4: HABITS & FEEDBACK LOOPS  
• Each evening: Rate your day (1–5) on focus & results. Note 1 win, 1 improvement.  
• Weekly review: adjust blocks, drop low-value tasks.  
• Apply 2-minute rule: if a task ≤2 min, do now; otherwise defer into a block.  
• Use time-boxing: Parkinson’s Law—give yourself tight windows.  

SYSTEMS THINKING TIP  
Your calendar is your system’s operating model. By protecting Q2 blocks (growth and strategy), you create a virtuous cycle: more skill → higher leverage → fewer crises. Over time, Q1 shrinks as you solve root causes rather than just urgent fires.  

—  
With this approach you shift from “busy” to “purposeful,” ensuring each day moves you forward on what truly matters.

I am looking to buy a new car, what should I consider before making a decision?

When buying a car you’re making a medium- to long-term commitment that ripples through your finances, daily routines and even your environmental footprint.  To make a high‐leverage choice, follow a structured, systems-thinking approach:

1. CLARIFY YOUR NEEDS & CONSTRAINTS  
   • Purpose: commute, family hauler, weekend fun, off-road, business use?  
   • Timing: How soon do you need delivery? Are you willing to wait for incentives or new model releases?  
   • Budget: total purchase price and/or monthly payment you can comfortably afford.  
   • Financing vs. cash: loan interest, lease vs. buy trade-offs.  
   • Non-negotiables: seating capacity, cargo volume, towing capability, ground clearance, drivetrain.  

2. DEFINE & WEIGH YOUR DECISION CRITERIA  
   List all factors that matter and assign each a relative weight (total = 100%). Typical criteria:  
   • Affordability/monthly payment  
   • Total cost of ownership (insurance, maintenance, fuel/charging)  
   • Reliability & warranty coverage  
   • Safety ratings & driver-assist features  
   • Fuel economy or energy efficiency  
   • Environmental impact (emissions, lifecycle footprint)  
   • Depreciation/resale value  
   • Comfort & convenience features  
   • Brand/dealer reputation  

3. BUILD A WEIGHTED DECISION MATRIX  
   Populate with the makes/models you’re considering. Rate each 1–10 against each criterion, multiply by the weight, and sum. Whichever scores highest is your top candidate.

   | Criteria                         | Weight (%) | Model A Score | A × W | Model B Score | B × W | Model C Score | C × W |
   |----------------------------------|-----------:|--------------:|------:|--------------:|------:|--------------:|------:|
   | Affordability / Monthly Payment  |         20 |             8 |   1.6 |             6 |   1.2 |             7 |   1.4 |
   | Total Cost of Ownership          |         15 |             7 |  1.05 |             8 |   1.2 |             6 |   0.9 |
   | Reliability & Warranty           |         15 |             9 |  1.35 |             7 |  1.05 |             8 |   1.2 |
   | Safety & Driver-Assist           |         15 |             8 |   1.2 |             9 |  1.35 |             7 |  1.05 |
   | Fuel Economy / Efficiency        |         10 |             7 |   0.7 |             9 |   0.9 |             6 |   0.6 |
   | Environmental Impact             |          5 |             6 |   0.3 |             8 |   0.4 |             5 |   0.25|
   | Depreciation / Resale Value      |         10 |             8 |   0.8 |             7 |   0.7 |             6 |   0.6 |
   | Comfort & Features               |         10 |             7 |   0.7 |             8 |   0.8 |             9 |   0.9 |
   | **Total**                        |       **100** |              | **7.7**|              | **7.6**|              | **6.9**|

   – In this example, Model A narrowly leads.  You’d focus on test-driving A & B, negotiating on A, and verifying final costs.

4. RISK & SYSTEMS THINKING CHECK  
   • Supply‐chain/trade-in uncertainties: are incentives at risk of expiring?  
   • Maintenance network: is service widely available in your area?  
   • Future needs: will your family grow? Are zoning/emissions rules changing?  
   • Feedback loop: after 6 months, re-assess your satisfaction vs. expectations.  

5. VALIDATE & FINALIZE  
   1. Test drive your top 2–3 picks in real conditions (highway, city, cargo load).  
   2. Get insurance quotes on each.  
   3. Compare dealer vs. private-sale vs. certified-pre-owned pricing.  
   4. Negotiate financing/april offers and add-on costs.  
   5. Lock in the deal only when all criteria and weighted scores align with your budget and long-term goals.

By making your decision criteria explicit, quantifying the trade-offs and building feedback loops, you transform a “gut choice” into a repeatable, data-driven system that ensures you buy the right car—for now and the road ahead.